In [ ]:
! pip install sentencepiece

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-train-multilingual-coments-google-api/jigsaw-toxic-comment-train-google-pt.csv
/kaggle/input/jigsaw-train-multilingual-coments-google-api/jigsaw-toxic-comment-train-google-ru.csv
/kaggle/input/jigsaw-train-multilingual-coments-google-api/jigsaw-toxic-comment-train-google-fr-cleaned.csv
/kaggle/input/jigsaw-train-multilingual-coments-google-api/jigsaw-toxic-comment-train-google-pt-cleaned.csv
/kaggle/input/jigsaw-train-multilingual-coments-google-api/jigsaw-toxic-comment-train-google-it.csv
/kaggle/input/jigsaw-train-multilingual-coments-google-api/jigsaw-toxic-comment-train-google-it-cleaned.csv
/kaggle/input/jigsaw-train-multilingual-coments-google-api/jigsaw-toxic-comment-train-google-ru-cleaned.csv
/kaggle/input/jigsaw-train-multilingual-coments-google-api/jigsaw-toxic-comment-train-google-es-cleaned.csv
/kaggle/input/jigsaw-train-multilingual-coments-google-api/jigsaw-toxic-comment-train-google-fr.csv
/kaggle/input/jigsaw-train-multilingual-coments-google-api/j

In [2]:
MAX_LEN = 128
BATCH_SIZE = 16 # per TPU core
TOTAL_STEPS = 2000  # thats approx 4 epochs
EVALUATE_EVERY = 200
LR =  1e-5

PRETRAINED_MODEL = 'jplu/tf-xlm-roberta-large'
D = '/kaggle/input/jigsaw-multilingual-toxic-comment-classification/'


import os
import numpy as np
import pandas as pd
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.optimizers import Adam
import transformers
from transformers import TFAutoModelWithLMHead, AutoTokenizer
import logging
# no extensive logging 
logging.getLogger().setLevel(logging.NOTSET)

AUTO = tf.data.experimental.AUTOTUNE

E0000 00:00:1725972869.632657    1297 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
D0910 12:54:29.640925574    1297 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0910 12:54:29.640939588    1297 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0910 12:54:29.640942735    1297 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0910 12:54:29.640945084    1297 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0910 12:54:29.640947490    1297 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defau

2.16.1


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def connect_to_TPU():
    """Detect hardware, return appropriate distribution strategy"""
    try:
        # TPU detection. No parameters necessary if TPU_NAME environment variable is
        # set: this is always the case on Kaggle.
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
    else:
        # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
        strategy = tf.distribute.get_strategy()

    global_batch_size = BATCH_SIZE * strategy.num_replicas_in_sync

    return tpu, strategy, global_batch_size


tpu, strategy, global_batch_size = connect_to_TPU()
print("REPLICAS: ", strategy.num_replicas_in_sync)


Running on TPU  
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1725972877.639986    1297 service.cc:145] XLA service 0x5b2b15d40610 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1725972877.640052    1297 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1725972877.640057    1297 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1725972877.640060    1297 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1725972877.640066    1297 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1725972877.640069    1297 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1725972877.640071    1297 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1725972877.640074    1297 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1725972877.640077    1297 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


REPLICAS:  8


In [4]:
D_TRANS = '/kaggle/input/jigsaw-train-multilingual-coments-google-api/'

def load_jigsaw_trans(langs=['tr','it','es','ru','fr','pt'], 
                      columns=['comment_text', 'toxic']):
    train_6langs=[]
    for i in range(len(langs)):
        fn = D_TRANS+'jigsaw-toxic-comment-train-google-%s-cleaned.csv'%langs[i]
        train_6langs.append(downsample(pd.read_csv(fn)[columns]))

    return train_6langs

def downsample(df):
    """Subsample the train dataframe to 50%-50%"""
    ds_df= pd.concat([
        df.query('toxic==1'),
        df.query('toxic==0').sample(sum(df.toxic))
    ])
    
    return ds_df
    

train_df = pd.concat(load_jigsaw_trans()) 

In [5]:
train_df.comment_text.values

array(['KADIN ÇALIŞMAYA ÇALIŞMADAN ÖNCE COCKSUCKER',
       'Üzgünüm\n\nBirinin konuşma sayfasıyla uğraştığım için üzgünüm. Bunu yapmak çok kötüydü. Konuşma sayfalarındaki şablonlara sahip olmanın, onlara hakimiyetinizi kanıtlamanıza nasıl yardımcı olduğunu biliyorum. Yüce yöneticilere boyun eğmem gerektiğini biliyorum. Ama sonra tekrar, dışarıda oynamaya gideceğim .... annenle. 76.122.79.82',
       'Göt içinde pis annesini sik, kuru!', ...,
       'Se ele faz uma proposta razoável sobre como alternar o texto, não tenho nenhum problema. No entanto, acredito firmemente que o objetivo oficial da organização deve ser listado na caixa de informações, e não no que alguns outros têm a dizer sobre eles.',
       '"\n\nDesculpe cara, se meu comportamento é rude. Mas estou pensando de maneira oposta, você está pensando. Na verdade, eu estou em uma imaginação que seção de chumbo deve ser pequena. Anteriormente, você me mostrou um bom exemplo de artigo. Mas, de certa forma, não estou feliz com s

In [6]:
%%time

def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])
    
PRETRAINED_TOKENIZER=  'jplu/tf-xlm-roberta-large'

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_TOKENIZER)
X_train = regular_encode(train_df.comment_text.values.tolist(), tokenizer, maxlen=MAX_LEN)
# X_test = regular_encode(test_df.content.values, tokenizer, maxlen=MAX_LEN)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /jplu/tf-xlm-roberta-large/resolve/main/tokenizer_config.json HTTP/11" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /jplu/tf-xlm-roberta-large/resolve/main/config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /jplu/tf-xlm-roberta-large/resolve/main/tokenizer_config.json HTTP/11" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /jplu/tf-xlm-roberta-large/resolve/main/sentencepiece.bpe.model HTTP/11" 200 0
/usr/local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31

CPU times: user 9min 34s, sys: 33.5 s, total: 10min 7s
Wall time: 30.4 s


In [7]:
def prepare_mlm_input_and_labels(X):
    # 15% BERT masking
    inp_mask = np.random.rand(*X.shape)<0.15 
    # do not mask special tokens
    inp_mask[X<=2] = False
    # set targets to -1 by default, it means ignore
    labels =  -1 * np.ones(X.shape, dtype=int)
    # set labels for masked tokens
    labels[inp_mask] = X[inp_mask]
    
    # prepare input
    X_mlm = np.copy(X)
    # set input to [MASK] which is the last token for the 90% of tokens
    # this means leaving 10% unchanged
    inp_mask_2mask = inp_mask  & (np.random.rand(*X.shape)<0.90)
    X_mlm[inp_mask_2mask] = 250001  # mask token is the last in the dict

    # set 10% to a random token
    inp_mask_2random = inp_mask_2mask  & (np.random.rand(*X.shape) < 1/9)
    X_mlm[inp_mask_2random] = np.random.randint(3, 250001, inp_mask_2random.sum())
    
    return X_mlm, labels


# use validation and test data for mlm
X_train_mlm = np.vstack([X_train])
# masks and labels
X_train_mlm, y_train_mlm = prepare_mlm_input_and_labels(X_train_mlm)


In [8]:
def create_dist_dataset(X, y=None, training=False):
    dataset = tf.data.Dataset.from_tensor_slices(X)

    ### Add y if present ###
    if y is not None:
        dataset_y = tf.data.Dataset.from_tensor_slices(y)
        dataset = tf.data.Dataset.zip((dataset, dataset_y))
        
    ### Repeat if training ###
    if training:
        dataset = dataset.shuffle(len(X)).repeat()

    dataset = dataset.batch(global_batch_size).prefetch(AUTO)

    ### make it distributed  ###
    dist_dataset = strategy.experimental_distribute_dataset(dataset)

    return dist_dataset
    
    
train_dist_dataset = create_dist_dataset(X_train_mlm, y_train_mlm, True)

In [9]:
%%time

def create_mlm_model_and_optimizer():
    with strategy.scope():
        model = TFAutoModelWithLMHead.from_pretrained(PRETRAINED_MODEL)
        optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
    return model, optimizer


mlm_model, optimizer = create_mlm_model_and_optimizer()
mlm_model.summary()

/usr/local/lib/python3.10/site-packages/transformers/models/auto/modeling_tf_auto.py:721: FutureWarning: The class `TFAutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `TFAutoModelForCausalLM` for causal language models, `TFAutoModelForMaskedLM` for masked language models and `TFAutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /jplu/tf-xlm-roberta-large/resolve/main/config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /jplu/tf-xlm-roberta-large/resolve/main/model.safetensors HTTP/11" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /jplu/tf-xlm-roberta-large/resolve/main/tf_model.h5 HTTP/11" 302 0
DEBUG:filelock:Attempting to acquire lock 135357977753776 on /root/.cache/huggingface/hub/.locks/models--jplu--tf-xlm-roberta-large/bd502c50deb6efcb00fd91768ac2a933716d50b8d85b1fcad21b7d3116ed25c7.lock
DEBUG:filelock:Lock 

Model: "tfxlm_roberta_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFXLMRobertaMainL  multiple                  558840832 
 ayer)                                                           
                                                                 
 lm_head (TFXLMRobertaLMHea  multiple                  257833106 
 d)                                                              
                                                                 
Total params: 560142482 (2.09 GB)
Trainable params: 560142482 (2.09 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
CPU times: user 48.5 s, sys: 1min 15s, total: 2min 3s
Wall time: 1min 12s


In [10]:
def define_mlm_loss_and_metrics():
    with strategy.scope():
        mlm_loss_object = masked_sparse_categorical_crossentropy

        def compute_mlm_loss(labels, predictions):
            per_example_loss = mlm_loss_object(labels, predictions)
            loss = tf.nn.compute_average_loss(
                per_example_loss, global_batch_size = global_batch_size)
            return loss

        train_mlm_loss_metric = tf.keras.metrics.Mean()
        
    return compute_mlm_loss, train_mlm_loss_metric


def masked_sparse_categorical_crossentropy(y_true, y_pred):
    y_true_masked = tf.boolean_mask(y_true, tf.not_equal(y_true, -1))
    y_pred_masked = tf.boolean_mask(y_pred, tf.not_equal(y_true, -1))
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true_masked,
                                                          y_pred_masked,
                                                          from_logits=True)
    return loss

            
            
def train_mlm(train_dist_dataset, total_steps=2000, evaluate_every=200):
    step = 0
    ### Training lopp ###
    for tensor in train_dist_dataset:
        distributed_mlm_train_step(tensor) 
        step+=1

        if (step % evaluate_every == 0):   
            ### Print train metrics ###  
            train_metric = train_mlm_loss_metric.result().numpy()
            print("Step %d, train loss: %.2f" % (step, train_metric))     

            ### Reset  metrics ###
            train_mlm_loss_metric.reset_states()
            
        if step  == total_steps:
            break


@tf.function
def distributed_mlm_train_step(data):
    strategy.run(mlm_train_step, args=(data,))


@tf.function
def mlm_train_step(inputs):
    features, labels = inputs

    with tf.GradientTape() as tape:
        predictions = mlm_model(features, training=True)[0]
        loss = compute_mlm_loss(labels, predictions)

    gradients = tape.gradient(loss, mlm_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, mlm_model.trainable_variables))

    train_mlm_loss_metric.update_state(loss)
    

compute_mlm_loss, train_mlm_loss_metric = define_mlm_loss_and_metrics()

In [ ]:
%%time
train_mlm(train_dist_dataset, TOTAL_STEPS, EVALUATE_EVERY)

DEBUG:absl:`TPUStrategy.run` is called with [args: ((PerReplica:{
  0: <tf.Tensor 'data:0' shape=(16, 128) dtype=int64>,
  1: <tf.Tensor 'data_1:0' shape=(16, 128) dtype=int64>,
  2: <tf.Tensor 'data_2:0' shape=(16, 128) dtype=int64>,
  3: <tf.Tensor 'data_3:0' shape=(16, 128) dtype=int64>,
  4: <tf.Tensor 'data_4:0' shape=(16, 128) dtype=int64>,
  5: <tf.Tensor 'data_5:0' shape=(16, 128) dtype=int64>,
  6: <tf.Tensor 'data_6:0' shape=(16, 128) dtype=int64>,
  7: <tf.Tensor 'data_7:0' shape=(16, 128) dtype=int64>
}, PerReplica:{
  0: <tf.Tensor 'data_8:0' shape=(16, 128) dtype=int64>,
  1: <tf.Tensor 'data_9:0' shape=(16, 128) dtype=int64>,
  2: <tf.Tensor 'data_10:0' shape=(16, 128) dtype=int64>,
  3: <tf.Tensor 'data_11:0' shape=(16, 128) dtype=int64>,
  4: <tf.Tensor 'data_12:0' shape=(16, 128) dtype=int64>,
  5: <tf.Tensor 'data_13:0' shape=(16, 128) dtype=int64>,
  6: <tf.Tensor 'data_14:0' shape=(16, 128) dtype=int64>,
  7: <tf.Tensor 'data_15:0' shape=(16, 128) dtype=int64>
}),)

In [ ]:
mlm_model.save_pretrained('./')